## CS102 - Further Computing

Mark Howard<br>
School of Mathematical & Statistical Sciences<br>
NUI Galway<br>
mark.howard@nuigalway.ie

### 2. Aspects of Data Wrangling

# Week 7: Operating on Data Frames

* Some of the most interesting studies of data come from **combining different data sources**.

* These operations can involve anything from very straightforward **concatenation** of two different datasets, to more complicated database-style **joins and merges** that correctly handle any overlaps between the datasets.

* ``Series`` and ``DataFrame``s are built with this type of operation in mind, and `Pandas` includes functions and methods that make this sort of data wrangling fast and straightforward.

* `Pandas` inherits from `numpy` its UFuncs and efficient vectorized operations.

* `Pandas` supports **keeping the context** of data and **combining data** from different sources
  through **index preservation** and automatic **index alignment**.

* We will additionally see that there are well-defined operations between one-dimensional ``Series`` structures and two-dimensional ``DataFrame`` structures.

In [ ]:
import numpy as np
import pandas as pd

* The following class allows us to display multiple ``DataFrame``s side by side. 
* The code makes use of the special ``_repr_html_`` method, which `IPython` uses to implement its rich object display:

In [ ]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
    

## Index Preservation

* Any `NumPy` UFunc will work on `Pandas` objects.

In [ ]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4), index=list("june"))
ser

In [ ]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)),
                  columns=['A', 'B', 'C', 'D'])
df

* If we apply a UFunc on either of these objects, the result will be a new `Pandas` object **with the indices preserved**:

In [ ]:
np.exp(ser)

In [ ]:
np.sin(df * np.pi / 4)

## Index Alignment

* For binary operations on two ``Series`` or ``DataFrame`` objects, `Pandas` will align indices in the process of performing the operation.

### Index alignment in `Series`

* As an example, suppose we are combining two different data sources, and find only the top three US states by **area** and the top three US states by **population**:

In [ ]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
                  'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193,
                        'New York': 19651127}, name='population')
area

In [ ]:
population

* Let's see what happens when we divide these to compute the population density:

In [ ]:
population / area

* The resulting array contains the **union** of indices of the two input arrays.
* This union could be determined using standard `Python` **set arithmetic** on these indices:

In [ ]:
area.index

In [ ]:
population.index

In [ ]:
area.index | population.index

* Any item for which one or the other does not have an entry is marked with ``NaN``, or "Not a Number,".
* This is how `Pandas` marks missing data.

In [ ]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B

* One reason to prefer a `pandas` object method (like `add`) over the infix operator (like `+`) would be to specify a **fill value** for missing entries that avoids `NaN` entries in the result.

* For example, calling ``A.add(B)`` is equivalent to calling ``A + B``, but allows optional explicit specification of the fill value for any elements in ``A`` or ``B`` that might be missing:

In [ ]:
A.add(B, fill_value=0)

### Index alignment in `DataFrame`

A similar type of alignment takes place for **both columns and indices** when performing operations on `DataFrame`s:

In [ ]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)),columns=list('AB'));A

In [ ]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)),index=[0,2,1],columns=list('BAC'));B

In [ ]:
A + B #Indices are aligned correctly irrespective of their order in the two objects, and indices in the result are sorted.

* Indices are aligned correctly irrespective of their order in the two objects, and indices in the result are sorted.
* As was the case with `Series`, we can use the associated object's arithmetic method and pass any desired `fill_value` to be used in place of missing entries.
* Here we'll fill with the mean of all values in `A`:

In [ ]:
A.values # the underlying numpy array

In [ ]:
A.values.mean()

In [ ]:
fill = A.values.mean()
A.add(B, fill_value=fill)

The following table lists `Python` operators and their equivalent `Pandas` object methods:

| Python Operator | Pandas Method(s)                      |
|-----------------|---------------------------------------|
| ``+``           | ``add()``                             |
| ``-``           | ``sub()``, ``subtract()``             |
| ``*``           | ``mul()``, ``multiply()``             |
| ``/``           | ``truediv()``, ``div()``, ``divide()``|
| ``//``          | ``floordiv()``                        |
| ``%``           | ``mod()``                             |
| ``**``          | ``pow()``                             |


## Some Utility Functions

* We start by providing some functions that are useful for generating and displaying examples of data frames.

* This function creates a generic ``DataFrame`` that will be useful below:

In [ ]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind] for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

* The following class allows us to display multiple ``DataFrame``s side by side. 
* The code makes use of the special ``_repr_html_`` method, which `IPython` uses to implement its rich object display:

In [ ]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)
    

* See below for how this is used.

## Combining Datasets with `concat`

* Recall concatenation of `Numpy` arrays:

In [ ]:
x = [1, 2, 3];y = [4, 5, 6];z = [7, 8, 9];
np.concatenate([x, y, z])

* The first argument is a list or tuple of arrays to concatenate.
* Additionally, it takes an ``axis`` keyword that allows you to specify the axis along which the result will be concatenated:

In [ ]:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1)

* In `Pandas`, the function `pd.concat()` can be used for a simple concatenation of `Series` or `DataFrame` objects:

In [ ]:
ser1 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
ser2 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
pd.concat([ser1, ser2])

* This also works to concatenate `DataFrame`s:

In [ ]:
df1 = make_df('AB', [11, 12])
df2 = make_df('AB', [3, 5])
display('df1', 'df2', 'pd.concat([df1, df2])')

* Note how the indices of both `DataFrames` are preserved.
* By default, the concatenation takes place row-wise (i.e., ``axis=0``).
* Like ``np.concatenate``, ``pd.concat`` allows specification of an axis along which concatenation will take place:

In [ ]:
# There are no columns in common
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
display('df3', 'df4', "pd.concat([df3, df4], axis=0,sort=False)")

In [ ]:
# concatenate along axis=1 aka columns
display('df3', 'df4', "pd.concat([df3, df4], axis=1)")

### Inner and Outer Joins

* In the simple examples we just looked at, we were mainly concatenating ``DataFrame``s with shared column names.
* In practice, data from different sources might have different sets of column names, and ``pd.concat`` offers several options in this case.
* Consider the concatenation of the following two ``DataFrame``s, which have some (but not all!) columns in common:

In [ ]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
display('df5', 'df6', 'pd.concat([df5, df6],sort=False)')
#By default, the entries for which no data is available are filled with NaN values.

* By default, the entries for which no data is available are filled with `NaN` values.
* To change this, we can specify one of several options for the ``join`` and ``join_axes`` parameters of the concatenate function.
* By default, the join is a union of the input columns (``join='outer'``), but we can change this to an intersection of the columns using ``join='inner'``:

In [ ]:
display('df5', 'df6',
        "pd.concat([df5, df6], join='inner')")

## Database style Merge and Join

* One essential feature offered by `Pandas` is its high-performance, **in-memory join and merge** operations.

* The main interface for this is the `pd.merge` function, and we'll see few examples of how this can work in practice.

### Categories of Joins

* The ``pd.merge()`` function implements a number of types of joins: the **one-to-one**, **many-to-one**, and **many-to-many** joins.

* All three types of joins are accessed via an identical call to the ``pd.merge()`` interface; the **type of join** performed depends on the **form of the input data**.

### One-to-one joins

* As a concrete example, consider the following two ``DataFrame``s which contain information on several employees in a company:

In [ ]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']},
                  index=['A', 'B', 'C', 'D'])
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]},
                  index = [4,5,6,7])
display('df1', 'df2')

* The two tables have **different indices** and exactly **one column in common**.
* In both tables, each entry in that column is unique.
* Thus, the common column establishes a **one-to-one** matching between the rows of the two tables.
* To combine this information into a single ``DataFrame``, we can use the ``pd.merge`` function:

In [ ]:
display('df1', 'df2')

In [ ]:
df3 = pd.merge(df1, df2)
df3

In [ ]:
display('df1', 'df2','df3')

* The ``pd.merge()`` function recognizes that each ``DataFrame`` has an "employee" column, and automatically joins using this column as a **key**.
* The result of the merge is a new ``DataFrame`` that combines the information from the two inputs.
* Notice that the order of entries in each column is not necessarily maintained: in this case, the order of the "employee" column differs between ``df1`` and ``df2``, and the ``pd.merge()`` function correctly accounts for this.
* Additionally, the merge in general **discards the index**, except in the special case of merges by index.

### Many-to-one joins

* Many-to-one joins are joins in which one of the two key columns contains duplicate entries.
* The key columns thus establish a **many-to-one** relationship between the rows of the two tables.
* For the many-to-one case, the resulting ``DataFrame`` will preserve those duplicate entries as appropriate.

In [ ]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

In [ ]:
display('df3', 'df4', 'pd.merge(df3, df4)')

* Here the common (key) column is "group", which in the first table contains repeated columns.
* The merged `DataFrame` is a copy of that first table with an aditional column containing the "supervisor" information, where the information is repeated in one or more locations as required by the inputs.

### Many-to-many joins
* If the key column in both the left and right array contains duplicates, then the result is a many-to-many merge.

In [ ]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

* Note how in each column of the merged table, information is repeated as needed.

## Specification of the Merge Key

* It is also possible to use a combination of two or more columns as key for a merge
  (e.g. when last names are kept in a separate column from first names).

* By default, `pd.merge()` looks for one or more matching column names between the two inputs, and uses these as the key.

* **If the column names do not match up as they should**, 
  they can be specified explicitly, using appropriate keyword parameters (`on` or `left_on` or `right_on`)
  
* Also, an index can be specified as a key

## Inner Joins vs. Outer Joins

* By default, the merged table contains the **INtersection** of the values in the key columns of the two input tables;

* if a value appears only in one input table, it won't make it into the merged table; 

* this is known as an **INner join**.
* For example:


In [ ]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display('df6', 'df7', 'pd.merge(df6, df7)')

* Here the two datasets have only one "name" entry in common: Mary, so the merged table has only one row.

* We can specify the type of join explicitly by using the `how` keyword, which defaults to `"inner"`:

In [ ]:
pd.merge(df6, df7, how='inner')

* In contrast, an **outer join** returns a join over the union of the input columns, and fills in all missing values with `NaN`s:

In [ ]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

In [ ]:
# The outer join has many missing values, and dropping those rows returns the inner join
pd.merge(df6, df7, how='outer').dropna()

* Other options for the `how` keyword are `'left'`, and `'right'`.


* The **left join** and **right join** return joins over the left entries and right entries, respectively.

In [ ]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

In [ ]:
display('df6', 'df7', "pd.merge(df6, df7, how='right')")

## Example: US States Data

* Merge and join operations come up most often when combining data from different sources.
* Here we will consider an example of some data about US states and their populations.
* The data files can be found at http://github.com/jakevdp/data-USstates/:
* Let's take a look at the three datasets, using the Pandas ``read_csv()`` function:

In [ ]:
# Following are shell commands to download the data
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

In [ ]:
#head.() Returns the first n rows, default (n=5)
pop = pd.read_csv('data/state-population.csv');areas = pd.read_csv('data/state-areas.csv');abbrevs = pd.read_csv('data/state-abbrevs.csv')
display('pop.head()', 'areas.head()', 'abbrevs.head()')

* Given this information, say we want  **rank US states and territories by their 2010 population density**.

* Clearly the necessary data for this **relatively straightforward task** are  in these tables.

* But the tables need to be **matched and merged** in order to find the result.

* We'll start with a many-to-one merge that will give us the full state name within the population ``DataFrame``.
* We want to merge based on the ``state/region``  column of ``pop``, and the ``abbreviation`` column of ``abbrevs``.
* **N.B. Different column labels so we will need to assist `Pandas`** with `left_on` and `right_on`
* We'll use ``how='outer'`` to make sure no data is thrown away due to mismatched labels.

In [ ]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation')
merged.head()

In [ ]:
display('pop.head()', 'abbrevs.head()')

In [ ]:
merged = merged.drop('abbreviation', 1) # drop duplicate info. The 1 says to drop info from columns (0 for indices).
merged.head()

* Let's double-check whether there were any mismatches here, which we can do by looking for rows with nulls:

In [ ]:
merged.isnull().any()

* Some of the ``population`` info is null; let's figure out which these are!

In [ ]:
# This is a mask with true in the rows where population is null/NaN
merged['population'].isnull();

In [ ]:
# Use masking and .loc to return these rows of the dataframe
merged.loc[merged['population'].isnull()].head()

* It appears that all the null population values are from Puerto Rico prior to the year 2000; this is likely due to this data not being available from the original source. **We can't and won't really do anything about that**

* More importantly, we see also that some of the new ``state`` entries are also null, which means that there was no corresponding entry in the ``abbrevs`` key!
Let's figure out which regions lack this match:

In [ ]:
# This is the same as above really but focusing on 'state' column 
merged.loc[merged['state'].isnull()].head()

In [ ]:
# what values are in the state/region column?
merged.loc[merged['state'].isnull(), 'state/region'].unique()

* We can quickly infer the issue: our population data includes entries for Puerto Rico (PR) and the United States as a whole (USA), while these entries do not appear in the state abbreviation key.
* We can fix these quickly by filling in appropriate entries (Beware `==` versus `=`):

In [ ]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

* No more nulls in the ``state`` column: we're all set!

* Now we can merge the result with the area data using a similar procedure.
`pd.merge()` will use the common `state` column as key in both tables.

In [ ]:
# Base final df off the merged df, with additional info from the areas df
final = pd.merge(merged, areas, how='left')
final.head()

* Again, let's check for nulls to see if there were any mismatches:

In [ ]:
final.isnull().any()

* There are nulls in the ``area`` column; we can take a look to see which regions were ignored here:

In [ ]:
final['state'][final['area (sq. mi)'].isnull()].unique()

* We see that our ``areas`` ``DataFrame`` does not contain the area of the United States as a whole.

* We could insert the appropriate value (using the sum of all state areas, for instance), but in this case we'll just drop the null values because the population density of the entire United States is not relevant to our current discussion:

In [ ]:
print('Shape before inplace dropna',final.values.shape)
final.dropna(inplace=True) #The pandas core team discourages the use of the inplace parameter
print('Shape after inplace dropna',final.values.shape)
final.head()

* Now we have all the data we need. 
* To answer the question of interest, let's first select the portion of the data corresponding with the year 2010, and the total population.

In [ ]:
data2010 = final[(final.year == 2010) & (final.ages == 'total')]
data2010.head()

* Now let's compute the population density and display it in order.
* We'll start by re-indexing our data on the state, and then compute the result:

In [ ]:
data2010.set_index('state', inplace=True)
data2010.head()

In [ ]:
density = data2010.population / data2010['area (sq. mi)']
density.head()

In [ ]:
density.sort_values(ascending=False, inplace=True)
density.head()

* The result is a ranking of US states plus Washington, DC, and Puerto Rico in order of their 2010 population density, in residents per square mile.
* We can see that by far the densest region in this dataset is Washington, DC (i.e., the District of Columbia); among states, the densest is New Jersey.

* We can also check the end of the list:

In [ ]:
density.tail()

* The least dense state, by far, is Alaska, averaging slightly over one resident per square mile.

* This type of messy data merging is a common task when trying to answer questions using real-world data sources.

## References

### `pandas`
*  the ["Merge, Join, and Concatenate" section](http://pandas.pydata.org/pandas-docs/stable/merging.html) of the Pandas documentation

* `concat`: [[doc]](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html)
* Note: The following two lines produce the same output
* `merged.loc[merged['population'].isnull()]`
* `merged.query('population.isnull()', engine='python')`
* This second line uses `query`: [[doc]](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html)

## Exercises

1. Use the US data from above to rank US states and territories by their density in terms of people under the age of 18.

2. Use the US data from above to identify the three top and bottom US states and territories in terms
   of their under 18 population as a proportion of the total population.